<a href="https://colab.research.google.com/github/simranjita/Hybrid/blob/master/classification_lstm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from math import sqrt
from numpy import concatenate

from matplotlib a pyplot
from pandas import read_csv
from numpy import savetxt
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
from pandas import read_csv
from datetime import datetime

 

In [ ]:
# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install keras

In [ ]:
from keras.utils.np_utils import to_categorical


dataset = read_csv(r'/content/drive/My Drive/Share/ACRS-HSI/orgwgtKSCwo_0.csv',header=None)
dataset = np.array(dataset)
classes=dataset[:,-1]
dataset=np.delete(dataset,103,axis=1)
#classes = to_categorical(classes)


In [ ]:
from sklearn.decomposition import PCA
X=dataset
Xpca = PCA(n_components=100).fit_transform(X)

In [ ]:
a=np.zeros([5211,1])
for i in range(dataset.shape[0]):
  a[i]=i;
  
count=np.array(a)
datwc=np.hstack([count,Xpca])


array([[ 0.00000000e+00, -6.66316166e+02, -1.25106998e+02, ...,
         7.68765390e-01, -1.06628794e+00, -6.82999578e-01],
       [ 1.00000000e+00, -6.76219010e+02, -1.07266873e+02, ...,
         4.98102706e-01,  8.15168078e-01,  5.46011852e-01],
       [ 2.00000000e+00, -6.81673468e+02, -1.15131835e+02, ...,
        -3.20413844e-01, -1.27433800e+00,  7.22237421e-01],
       ...,
       [ 5.20800000e+03,  5.58956969e+04, -3.17004704e+04, ...,
         5.85024119e-02, -1.23565061e+00, -1.35422505e+00],
       [ 5.20900000e+03, -7.12550593e+02, -1.46815751e+02, ...,
         1.99719410e+00,  5.55869183e-01,  3.09584596e-01],
       [ 5.21000000e+03, -7.19297949e+02, -1.45553962e+02, ...,
        -2.64531160e-01, -5.31686634e-01,  2.79570532e-01]])

In [ ]:
dataset=datwc
from sklearn.model_selection import train_test_split

train_X, test_X, train_y, test_y = train_test_split(dataset,classes, test_size=0.50, random_state=42)
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

      


(2605, 1, 101) (2605,) (2606, 1, 101) (2606,)


In [ ]:
from keras.callbacks import ModelCheckpoint

# design network
model = Sequential()
model.add(LSTM(400, input_shape=(train_X.shape[1], train_X.shape[2]),dropout=0.0))
model.add(Dense(1,activation='softmax'))
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
filepath = "best-model.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]
model.summary()
# fit network
history = model.fit(train_X, train_y, validation_data=(test_X, test_y), epochs=10, verbose=1, callbacks=callbacks_list)
 
 

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 400)               803200    
_________________________________________________________________
dense (Dense)                (None, 1)                 401       
Total params: 803,601
Trainable params: 803,601
Non-trainable params: 0
_________________________________________________________________
Epoch 1/10
82/82 [==============================] - 24s 17ms/step - loss: nan - accuracy: 0.0075 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 2/10
82/82 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 3/10
82/82 [==============================] - 1s 7ms/step - loss: nan - accuracy: 0.0000e+00 - val_loss: nan - val_accuracy: 0.0000e+00
Epoch 4/10
82/82 [==============================] - 1s 7ms/step - loss: nan - accuracy: 

In [ ]:
# load best weights
model.load_weights("best-model.hdf5")
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
Y_pred_test = model.predict(test_X)
y_pred_test = np.argmax(Y_pred_test, axis=1)


In [ ]:

from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

print(classification_report(test_y,  y_pred_test))
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(test_y,  y_pred_test)
print(cm)
accuracy_score(test_y, y_pred_test)*100


              precision    recall  f1-score   support

           1       0.95      0.96      0.95       400
           2       0.85      0.90      0.87       111
           3       0.97      0.98      0.98       125
           4       0.99      0.73      0.84       132
           5       0.82      0.95      0.88        84
           6       0.97      0.88      0.92       117
           7       0.83      0.92      0.88        53
           8       0.96      0.98      0.97       218
           9       0.96      1.00      0.98       242
          10       0.96      0.92      0.94       198
          11       1.00      1.00      1.00       205
          12       0.96      0.94      0.95       247
          13       0.97      0.99      0.98       474
         189       0.00      0.00      0.00         0

    accuracy                           0.95      2606
   macro avg       0.87      0.87      0.87      2606
weighted avg       0.95      0.95      0.95      2606

[[384   1   0   0   2   2

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


95.28012279355333